In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Import Dependancies

In [1]:
#!/usr/bin/env python3.11
!python --version

import mesa
print(f"mesa version: {mesa.__version__}")
import numpy as np
import pandas as pd
import seaborn as sns
import random

Python 3.12.6
mesa version: 3.1.5


# The Naming Game

text

In [2]:
#ideas for own unique approach:
# multiple objects and graph how time until vocab convergence changes?
# different lengths of words and graph difference there?
# limit amount of symbols for words?
# 

Create Random Word

In [3]:
from random import choice
from string import ascii_uppercase

def create_word(min_length=6, max_length=10):
    new_word = ''.join(choice(ascii_uppercase) for _ in range(np.random.randint(min_length, max_length)))
    return new_word

Create Agent

In [4]:
class NamingGameAgent(mesa.Agent):
    def __init__(self, model, m_objects=1):
        super().__init__(model)
        self._m_objects = m_objects
        self.vocabulary = dict()

    def choose_object_and_word(self):
        #chosen from range will always choose 0 index object value, so m_objects = 1 -> chosen object is 0
        chosen_object = random.choice(range(self._m_objects))
        if chosen_object in self.vocabulary.keys():
            chosen_word = random.choice(self.vocabulary[chosen_object])
        else:
            chosen_word = create_word()
            self.vocabulary[chosen_object] = [chosen_word]
        return chosen_object, chosen_word

    def recognize_word(self, chosen_object, chosen_word):
        if chosen_object not in self.vocabulary.keys():
            return False
        if chosen_word in self.vocabulary[chosen_object]:
            return True
        else:
            return False

    def agree_to_word(self, chosen_object, chosen_word):
        self.vocabulary[chosen_object] = [chosen_word]

    def add_word(self, chosen_object, chosen_word):
        if chosen_object not in self.vocabulary.keys():
            self.vocabulary[chosen_object] = [chosen_word]
        else:
            self.vocabulary[chosen_object].append(chosen_word)

    def retrieve_vocabulary(self):
        return [(key, value) for key, value in self.vocabulary.items()]

Create Model

In [7]:
from mesa.datacollection import DataCollector

class NamingGameModel(mesa.Model):
    def __init__(self, N=100, m_objects=1, seed=None):
        super().__init__(seed=seed)
        self.num_agents = N
        #TODO figure out schedulte stuff self.schedule = mesa.time.RandomActivation(self) if needed
        #self.running = True
        #TODO make datacollection better
        self.datacollector = DataCollector(
            agent_reporters={"Vocabulary": NamingGameAgent.retrieve_vocabulary}
        )
        self.m_objects = m_objects
        NamingGameAgent.create_agents(model=self, n=N, m_objects=m_objects)
    
    def step(self):
        if self.steps % 100 == 0:
            self.datacollector.collect(self)
        speaker, hearer = random.sample(self.agents, 2)
        chosen_object, chosen_word = speaker.choose_object_and_word()
        if hearer.recognize_word(chosen_object, chosen_word):
            speaker.agree_to_word(chosen_object, chosen_word)
            hearer.agree_to_word(chosen_object, chosen_word)
        else:
            hearer.add_word(chosen_object, chosen_word)


Run Model

In [13]:
N = 1000
M = 1
t = 60000
model = NamingGameModel(N, M)
for i in range(t):
    #debug_vocab = model.agents.select(at_most=1).get("vocabulary")
    #if debug_vocab[0]:
    #    print(f"step {i}: {debug_vocab}")
    print(f"step: {i}")
        
    model.step()
    #TODO datacollection here and after all runs


agent_vocabulary = model.datacollector.get_agent_vars_dataframe()
one_agent_vocab = agent_vocabulary.xs(2, level="AgentID")
print(one_agent_vocab)
agent_vocabulary

step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
step: 34
step: 35
step: 36
step: 37
step: 38
step: 39
step: 40
step: 41
step: 42
step: 43
step: 44
step: 45
step: 46
step: 47
step: 48
step: 49
step: 50
step: 51
step: 52
step: 53
step: 54
step: 55
step: 56
step: 57
step: 58
step: 59
step: 60
step: 61
step: 62
step: 63
step: 64
step: 65
step: 66
step: 67
step: 68
step: 69
step: 70
step: 71
step: 72
step: 73
step: 74
step: 75
step: 76
step: 77
step: 78
step: 79
step: 80
step: 81
step: 82
step: 83
step: 84
step: 85
step: 86
step: 87
step: 88
step: 89
step: 90
step: 91
step: 92
step: 93
step: 94
step: 95
step: 96
step: 97
step: 98
step: 99
step: 100
step: 101
step: 102
step: 103
step: 104
step: 105
step: 106
step: 107
step: 108
step: 109
step: 110


Vocabulary
Step  AgentID                                                   
100   1                                                       []
      2        [(0, [CMMWAM, LWVUNVXZB, FFORVKVPD, LFFDRZV, K...
      3                                                       []
      4                                                       []
      5                                                       []
...                                                          ...
60000 996                                       [(0, [LFFDRZV])]
      997                                       [(0, [LFFDRZV])]
      998                                       [(0, [LFFDRZV])]
      999                                       [(0, [LFFDRZV])]
      1000                                      [(0, [LFFDRZV])]

[600000 rows x 1 columns]